# How does BPM correlate with song popularity over time? #
A computational essay by Richard Li, Natsuki Sacks, Norah Evans 

# Introduction #

Over the past few years of human existence, *everything* has gotten faster. Instant deliveries, messaging, food, etc. A common critique of the 21st century is that it's built on speed. Our group wondered if that holds true for songs. Have we begun enjoying faster songs as everything around us accelerates?

In [2]:
# Import code for scraping and cleaning data
# from billboard import *
# from billboard_functions import *
import requests
from bs4 import BeautifulSoup

# Methodology #

In order to answer our research question of whether the average tempo of the most popular songs has increased in the modern era, we must accomplish 3 things:

1. Find a metric for popularity among songs in a year
2. Find the BPM of those songs
3. Average the BPM and chart the trends over the course of many years

Each of these things occur in 3 files:
1. billboard.py
2. bpm.py
3. visualization.py

## Step One ##
We found this [Billboard Top 100](BillboardTop100of.com) website to scrape our data from. The Billboard Top 100 contains the rank, artist, and song for up to the top 100 songs for years 1940 to 2021. Some of the Top Billboards in the 40s contain only 30 to 80 songs, but beggars can't be choosers.

The metric for ranking on this website is based off of the Billboard Magazine, which has been keeping track of the music industry for years now.

### 1a. Scraping the Data ###
To analyze the data's BPM, we first need to scrape it from this website. We chose to scrape our data from the [Billboard Top 100](BillboardTop100of.com), since the APIs we found didn't provide all of the information we needed for free.

Each year contained a table made up of the song information for that year. To extract all of the information in that table (rank, artist, song), we used the Python Requests library to grab all of the data for a given year. We then parsed our data using BeautifulSoup, and then extracted all of the important information from that HTML parse by finding all data within a given tag (also by using Beautiful Soup).

In [ ]:
# The function we used to grab Billboard data for a given year.
def get_data(year):
    # Years 1940 and 2020 had different URLs than every other year 
    if year == (2020):
        url = "http://billboardtop100of.com/billboard-top-100-songs-2020-2/"
    elif year == 1940:
        url = "http://billboardtop100of.com/336-2/"
    else:
        url = f"http://billboardtop100of.com/{year}-2/"
    response = requests.get(url)
    data = BeautifulSoup(response.content, "html.parser")
    return data

We won't print out all of the data returns from the HTML Request, since it's pretty much a bunch of gibberish with some important information sprinkled in between. 

To figure out what tag we needed to look for, we went in to the [Billboard Top 100](BillboardTop100of.com) page for the given year and inspected an element that we needed (i.e. "The Weeknd"). We did end up doing this for about 15 pages or so, since years 1945 to 2016 had the same body tags, while all remaining years had varying tags. 

In [6]:
# how to make variables global?
# check 2013 and 2015 data
data = get_data(2014)
music_data = data.findAll("p")

print(music_data)

4


### 1b. Cleaning the Data ###
As we can see, `music_data` contains the information we need, but also contains a bunch of HTML code that we don't need. It also isn't formatted properly; at the end of all this, we want a list of lists that contain three string elements: the rank, artist, and song for all ~100 songs of the year.

For the more straightforward years, where each rank, artist, and song were already separated into a list of strings with random newlines or external links in between, the `remove_long_tags` function was used to clean all of that data by calling one line of code.

Some years had more convoluted tags, line breaks, external images, etc. stuck in between important information, so those were cleaned using the `remove_long_tags` function in addition to other `.replace` commands.

In [ ]:
# An example of what cleaning a less-messy data set looks like

# Removing all p-tags, "amp;", "/n", and more for 2014
clean_data = remove_long_tags("<p", music_data)

# Removes all external links (images, additional information)
clean_data_more = remove_long_tags("<a", clean_data)

# Splits the list of strings into a list of lists, grouping the strings into
# lists at intervals of three
split_list = split_list_into_three(clean_data_more)

print(split_list)

### 1c. Loading the Data into the Data Frame ***
`split_list` is then loaded into the data frame using the `import_to_df` function in `billboard_functions.py`. This data frame is 80 columns (one for each year searched for, excluding 2021) by 100 rows (one for every song on the Billboard Top 100 for each year).

In [ ]:
# The function used to load the files into the data frame
def import_to_df(split_list, year, df):
    for i in range(1, len(split_list) + 1):
        df.at[i, year] = split_list[i - 1]
        
    df.to_csv("../data/top100.csv")